# Maschinelles Lernen  
## Übung Termin 1 -  Datenaufbereitung

### Aufbau:
**1. Importieren von Daten**

    - Datensätze einladen
    - Zusammenführen und Filtern von Datensätze

**2. Datenaufbereitung:**

    - Ersetzen von Werten unterhalb der Nachweisgrenze
    - Ersetzen von kategorischen und numerischen *NaN* Einträgen
    - Feature Aggregation
    - Aufteilen in Test- und Train-Split
    - Skalieren (Normieren/Standardisieren)
    - Encodieren (Binär/One Hot/Target)
    - Speichern der Daten
 **3. Übungsaufgabe**

#### Import der Python Bibliotheken


In [1]:
# Basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Importieren der Daten.
## 1.1 Datensätze einladen

Wir verwenden zwei Datensätze
- Nitratdatensatz (Nitratmessungen an GW-Messstellen)
- [Corine Landnutzungarten](https://land.copernicus.eu/pan-european/corine-land-cover/clc2018) an den GW-Messstellen


Wir verwenden die Funktion <span style="color:blue">**pandas.read_csv**</span> &rarr; [Hilfe](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) um die Dateien zu importieren.

### 1.1.1 Nitratdatensatz:

In [2]:
# Nitratdatensatz einlesen
data_NO3 = pd.read_csv('Nitratmessungen.csv', sep=',', encoding="utf-8", index_col=0)

# Zeige die ersten 5 Zeilen des Datensatzes an
data_NO3.head(5)

,GW-Nummer,Messstelle,Datum,NO3 [mg/l],pH [ -],O2 [mg/l],O2-Sätt [%],T [°C],RECHTSWERT,HOCHWERT,HYDROGEOLE,AQUIFER,HYDROGEOL3
0,918/069-2,"BBR Betonwerk, Umkirch",11.10.2006 15:20,"9,4","6,45","2,4","23,3","14,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
1,918/069-2,"BBR Betonwerk, Umkirch",17.09.2007 14:20,12,"6,39","2,1","25,0","21,8",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
2,918/069-2,"BBR Betonwerk, Umkirch",23.09.2008 08:05,"7,6","6,58","2,5","23,0",NaN,3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
3,918/069-2,"BBR Betonwerk, Umkirch",10.11.2008 09:10,"12,2","6,54","1,2","12,0","15,4",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)
4,918/069-2,"BBR Betonwerk, Umkirch",09.09.2009 14:08,12,"6,46","1,3","14,0","20,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL)


In [3]:
# Gebe Datentypen für die einzelnen Spalten aus
# Bsp. int64, float64, bool, datetime64, timedelta, object, usw.
data_NO3.dtypes

GW-Nummer       object
Messstelle      object
Datum           object
NO3 [mg/l]      object
pH [ -]         object
O2 [mg/l]       object
O2-Sätt [%]     object
T [°C]          object
RECHTSWERT       int64
HOCHWERT         int64
HYDROGEOLE     float64
AQUIFER         object
HYDROGEOL3      object
dtype: object

> **Vorsicht:** Hier sehen wir, dass für mehrere Spalten falsche Datentypen vorliegen. Die Spalte `Datum` sollte anstelle von `Object` als Datetimeformat vorliegen. Die Spalten mit den physikochemischen Eigenschaften und dem Nitratgehalt sollten auch in `Float` umgewandelt werden, damit Berechnungen durchgeführt werden können. Diese Änderungen werden wir später in der Datenaufbereitung durchführen.


### 1.1.2 Corine Landnutzung für Messstellen:

Die CORINE Landnutzungsdaten der Europäischen Umweltagentur klassifizieren die Landnutzung in Europa. Sie umfassen Kategorien wie Ackerland (Code 1), Wälder (Code 2), Siedlungsgebiete (Code 3) und andere. Diese Daten werden in Form von Raster- oder Vektordaten bereitgestellt, oft in einem rasterbasierten Format mit spezifischer räumlicher Auflösung. In der Datei 'Corinedaten.txt' wurden in einem Vorschritt die Landnutzungsklassen direkt an den Standorten der Messstellen erfasst.

In [4]:
# Corine Datensatz einlesen
data_Corine = pd.read_csv('Corinedaten.txt', sep=';',decimal=',',  encoding='utf-8', index_col=0)

# Zeige die ersten 5 Zeilen des Datensatzes an
data_Corine.head(5)

,GW_NUMMER,NAME,RECHTSWERT,HOCHWERT,GEMEINDE_K,AQUIFER_KZ,EINZUGSGEB,HYDROGEOLE,HYDROGEOL2,MESSNETZ1_,MESSNETZ2_,GEMEINDE,AQUIFER,EINZUGSGE2,HYDROGEOL3,MESSNETZ1,MESSNETZ2,RASTERVALU
FID,,,,,,,,,,,,,,,,,,
0,1/022-9,"QSS HERTINGERQU.1+2, KANDERN",3395475.0,5288420.0,8336104.0,31.0,2.333220e+09,11.0,2.0,,,MALSBURG-MARZELL,MALM WEIßJURA (SCHWÄBISCHE ALB),Engebach,Oberjura (Raurasische Fazies) (GWL),#800000000010,#800000000002,311
1,1/117-3,"QF KREBSBRUNNENQ., ETTENHEIM",3419400.0,5345550.0,8317026.0,81.0,2.338940e+09,45.0,69.0,,,"ETTENHEIM, STADT",BUNTSANDSTEIN,Ettenbach,Unterer und Mittlerer Buntsandstein (GWL),#800000000010,#800000000002,311
2,1/119-9,"TB SCHLAGBR.4, VOERSTETTEN",3414860.0,5325450.0,8316045.0,4.0,2.338892e+09,3.0,8.0,,,VÖRSTETTEN,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Schobbach,Quartäre Kiese und Sande (GWL),#800000000012,,211
3,1/120-8,"QF2 KLEISLEWALDQ., ZASTLER",3423200.0,5309100.0,8315084.0,91.0,2.338830e+09,50.0,18.0,,,OBERRIED,KRISTALLIN (SCHWARZWALD),Dreisam unterh. Wagensteigbach oberh. Brugga,"Paläozoikum, Kristallin (GWG)",#800000000010,#800000000002,312
4,1/121-0,"Q STOLLENQUELLE HINTERES ELEND, MÜNSTERTAL",3413785.0,5303338.0,8315130.0,91.0,2.336410e+09,50.0,18.0,,,MÜNSTERTAL/ SCHWARZWALD,KRISTALLIN (SCHWARZWALD),Neumagen oberhalb von Talbach,"Paläozoikum, Kristallin (GWG)",#800000000010,#800000000002,231


## 1.2. Zusammenführen und Filtern der Datensätze

Zusammenführen der beiden Datensätze mit Hilfe der <span style="color:blue">**.merge**</span>-Funktion für Pandas Dataframes. &rarr; [Hilfe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)
- Gemeinsame Spalte für das Zusammenführen ist die **Grundwassernummer**.

- Von dem zweiten Datensatz benötigen wir lediglich die Spalte **"RASTERVALU"**, diese enthält Daten zur Landnutzung von der Corine Landuse.

### 1.2.1 Zusammenführen der Datensätze

> **Info:** Für die weitere Verarbeitung benötigen wir von den Corinedaten nur die Spalte "Rastervalue" und die "GW-Nummer" als Schlüssel, um beim Zusammenführen die richtigen Werte zuzuweisen.


In [5]:
# Führe die beiden Datensätze anhand der gemeinsamen Spalte "GW-Nummer" zusammen und behalte nur die benötigten Spalten
joined_data = data_NO3.merge(data_Corine[['GW_NUMMER','RASTERVALU']], how='inner', left_on='GW-Nummer', right_on='GW_NUMMER')

# Zeige die ersten 5 Zeilen des zusammengeführten Datensatzes an
joined_data.head(5)

,GW-Nummer,Messstelle,Datum,NO3 [mg/l],pH [ -],O2 [mg/l],O2-Sätt [%],T [°C],RECHTSWERT,HOCHWERT,HYDROGEOLE,AQUIFER,HYDROGEOL3,GW_NUMMER,RASTERVALU
0,918/069-2,"BBR Betonwerk, Umkirch",11.10.2006 15:20,"9,4","6,45","2,4","23,3","14,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
1,918/069-2,"BBR Betonwerk, Umkirch",17.09.2007 14:20,12,"6,39","2,1","25,0","21,8",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
2,918/069-2,"BBR Betonwerk, Umkirch",23.09.2008 08:05,"7,6","6,58","2,5","23,0",NaN,3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
3,918/069-2,"BBR Betonwerk, Umkirch",10.11.2008 09:10,"12,2","6,54","1,2","12,0","15,4",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121
4,918/069-2,"BBR Betonwerk, Umkirch",09.09.2009 14:08,12,"6,46","1,3","14,0","20,0",3407870,5323120,3.0,QUARTÄR EISZEITL.KIESE+SANDE (OBERRHEINGRABEN),Quartäre Kiese und Sande (GWL),918/069-2,121


In [6]:
# Gebe Datentypen für die einzelnen Spalten des zusammengeführten Dataframes aus
joined_data.dtypes

GW-Nummer       object
Messstelle      object
Datum           object
NO3 [mg/l]      object
pH [ -]         object
O2 [mg/l]       object
O2-Sätt [%]     object
T [°C]          object
RECHTSWERT       int64
HOCHWERT         int64
HYDROGEOLE     float64
AQUIFER         object
HYDROGEOL3      object
GW_NUMMER       object
RASTERVALU       int64
dtype: object

### 1.2.2 Filtern der Datensätze

#### Für die weitere Verarbeitung benötigen wir nur noch die folgenden Spalten:
- Messstelle
- GW-Nummer 
- Datum
- NO3
- O2
- Rastervalu = Corine Landcover
- Hydrogeologie

In [7]:
# Erstelle DataFrame mit ausgewählten Spalten (columns) und konvertiere 'Datum' in ein Datetime-Objekt
selected_columns = ['Messstelle', 'GW-Nummer', 'Datum', 'NO3 [mg/l]', 'O2 [mg/l]', 'RASTERVALU', 'HYDROGEOL3']
data = joined_data[selected_columns].copy()  # selected_columns auswählen und eine Kopie erstelle

# 'Datum' in ein Datetime-Objekt konvertieren
data['Datum'] = pd.to_datetime(data['Datum'], format='%d.%m.%Y %H:%M')  

# Zeige die ersten 5 Zeilen des zusammengeführten Datensatzes an
data.head(5)
data.dtypes

Messstelle            object
GW-Nummer             object
Datum         datetime64[ns]
NO3 [mg/l]            object
O2 [mg/l]             object
RASTERVALU             int64
HYDROGEOL3            object
dtype: object

> **Vorsicht:** Normalerweise könnten wir die Objekte mit `.astype(float)` einfach umwandeln. Wenn ihr das versucht, werdet ihr eine Fehlermeldung erhalten. Diese liegt daran, dass die Daten ein Komma statt eines Punktes als Dezimaltrennzeichen besitzen. Daher müssen wir das Komma mit einem Punkt ersetzen.
Es lohnt sich daher die Spalten ein wenig genauer zu betrachten.

# 2. Datenaufbereitung

> Hinweis: Bevor ihr Manipulationen am DataFrame durchführen, kann es hilfreich sein, eine Kopie des DataFrame zu erstellen,
um mögliche Fehlermeldungen zu vermeiden, insbesondere wenn Ihr Zellen mehrmals ausführt. 
Ihr könnt dies mit dem Befehl `df = data.copy()` tun. 
Auf diese Weise müsst ihr nicht das gesamte Skript erneut ausführen, sondern können einfach ab dieser Zelle fortsetzen.


In [8]:
# Erstelle eine Kopie des DataFrame
df= data.copy()

Spalte `"NO3 [mg/l]"`

In [9]:
# Sortiere die Werte der Spalte in aufsteigender Reihenfolge
df["NO3 [mg/l]"].sort_values() # --> Beeinhaltet Werte unterhalb der Nachweisgrenze (NWG)

5026      0,1
1707      0,1
5873      0,1
5543      0,2
6549      0,2
        ...  
8294    < 0,5
1366    < 0,5
1367    < 0,5
8248    < 0,5
5724    < 0,5
Name: NO3 [mg/l], Length: 12178, dtype: object

> **Vorsicht:** Wir sehen hier, dass es Werte gibt, die unterhalb der Nachweisgrenze liegen. Diese wurden mit "<" gekennzeichnet. Da "Pandas" nicht mit "<" arbeiten kann, müssen wir a) die Zeichen entfernen und b) überlegen, was wir mit den Werten unterhalb der Nachweisgrenze machen.


Spalte `"O2 [mg/l]"`

In [10]:
# Sortiere die Werte der Spalte in aufsteigender Reihenfolge
df["O2 [mg/l]"].sort_values() #--> beeinhaltet NAN Werte

8181     0,1
3490     0,1
5548     0,1
3485     0,1
685      0,1
        ... 
12160    NaN
12161    NaN
12162    NaN
12163    NaN
12164    NaN
Name: O2 [mg/l], Length: 12178, dtype: object

> **Vorsicht:** Wir haben festgestellt, dass NaN-Werte im Datensatz vorhanden sind. Möglicherweise wurden diese Parameter nicht gemessen. In vielen Fällen müssen daher fehlende Werte geschätzt oder imputiert werden.
> Eine allgemein akzeptierte Regel, wie viele Werte maximal geschätzt werden dürfen, ist schwer zu bestimmen, da sie von verschiedenen Faktoren abhängt, wie z.B. der Art der Daten, der Qualität der vorhandenen Daten, der Analysemethode und dem Anwendungsfall.
> Jedoch wird oft empfohlen, dass nicht mehr als 5-10% der Daten imputiert werden sollten, um die Zuverlässigkeit der Ergebnisse zu gewährleisten.

In [11]:
# FYI: Anzahl von "<" und "NAN" ausgeben
# Spaltennamen definieren
columns = ['O2 [mg/l]', 'NO3 [mg/l]']

# Schleife durch die Spaltennamen
for col in columns:
    # Anzahl von '<'-Zeichen zählen
    count_lessthan = df[col].str.count('<').sum()
    # Anzahl von NaN-Werten zählen
    count_nan = df[col].isna().sum()
    # Gesamtzahl der Einträge in der Spalte
    total_entries = len(df)
    
    # Prozentsatz von '<'-Zeichen berechnen
    percent_lessthan = round((count_lessthan / total_entries) * 100, 2)
    # Prozentsatz von NaN berechnen
    percent_nan = round((count_nan / total_entries) * 100, 2)
    
    # Ausgabe
    print(f"In der Spalte '{col}' gibt es {count_lessthan} '<' Zeichen ({percent_lessthan}%) und {count_nan} NaN-Werte ({percent_nan}%).")


In der Spalte 'O2 [mg/l]' gibt es 1413.0 '<' Zeichen (11.6%) und 475 NaN-Werte (3.9%).
In der Spalte 'NO3 [mg/l]' gibt es 2110 '<' Zeichen (17.33%) und 0 NaN-Werte (0.0%).


### 2.1 Ersetzen von Werten unterhalb der Nachweisgrenze (NWG)

Die NWG (Nachweisgrenze) ist der kleinste messbare Wert eines Analyten in einem bestimmten System oder Labor. Werte unterhalb der NWG gelten als nicht signifikant.

In diesem Schritt werden diese Werte durch ´0,5 * NWG´ ersetzt, um eine grobe Schätzung des tatsächlichen Wertes zu erhalten, basierend auf der Annahme, dass der wahre Wert irgendwo zwischen 0 und der NWG liegt.

*Es ist wichtig zu beachten, dass diese Art der Behandlung von Werten unterhalb der NWG nicht immer angemessen oder geeignet ist. In einigen Fällen kann es besser sein, die Daten auszulassen oder alternative Methoden zur Schätzung der wahren Werte zu verwenden.*


In [12]:
def ersetze_unter_NWG(wert):
    """
    Funktion, die Werte unterhalb der NWG durch 0.5*NWG ersetzt.
    Args:
    wert (int/float/str): Wert, entweder ein Integer, Float oder String.
    Returns:
    Eingabewert, wenn er größer/gleich NWG,
    0.5*NWG, wenn der Wert kleiner NWG.
    """
        # Wenn der Wert ein String ist, wird er in einen float umgewandelt.
    if isinstance(wert, str):
        # Überprüfen, ob der String ein "<" enthält, was auf eine Bereichsangabe hinweist.
        if '<' in wert:
            # Extrahiere den Bereichswert und dividiere ihn durch 2.
            bereichswert = float(wert.split('<')[1].replace(',', '.')) / 2
        else:
            # Wenn kein "<" vorhanden ist, wird der String in einen float umgewandelt.
            bereichswert = float(wert.replace(',', '.'))
    else:
        # Wenn der Wert kein String ist, wird er direkt verwendet.
        bereichswert = wert
    # Gib den finalen Wert zurück.
    return bereichswert


Anwenden unserer Funktion <span style="color:green">**ersetze_unter_NWG**</span>-Funktion
auf die Spalten `'NO3 [mg/l]` und `O2 [mg/l]`*, um die neuen Spalten `NO3` und `O2` zu erstellen.  

Verwendete Funktionen:

<span style="color:blue">**.apply**</span>-Funktion &rarr; [Hilfe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)

<span style="color:blue">**.drop**</span>-Funktion &rarr; [Hilfe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)

Anschließendes Entfernen der ehemaligen Spalten.



In [13]:
# Erstelle neue Spalten "O2" und "NO3" durch Anwenden von ersetze_unter_NWG auf die Spalten "O2 [mg/l]" und "NO3 [mg/l]"
df['O2'] = df['O2 [mg/l]'].apply(ersetze_unter_NWG)
df['NO3'] = df['NO3 [mg/l]'].apply(ersetze_unter_NWG)

# Lösche die alten Spalten "O2 [mg/l]" und "NO3 [mg/l]"
df.drop(columns=['O2 [mg/l]', 'NO3 [mg/l]'], inplace=True)

# Sortiere den DataFrame nach aufsteigendem NO3-Wert und gib die ersten fünf Zeilen aus
df.head()

,Messstelle,GW-Nummer,Datum,RASTERVALU,HYDROGEOL3,O2,NO3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,121,Quartäre Kiese und Sande (GWL),2.4,9.4
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,121,Quartäre Kiese und Sande (GWL),2.1,12.0
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,121,Quartäre Kiese und Sande (GWL),2.5,7.6
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,121,Quartäre Kiese und Sande (GWL),1.2,12.2
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,121,Quartäre Kiese und Sande (GWL),1.3,12.0


### 2.2 Ersetzen von fehlenden Werten (NULL-Values)
Datensätze sind oft unvollständig und enthalten fehlende Werte, sog: `NULL` values oder `NaNs`. Diese können verschiedene Ursachen haben, z.B. unvollständige Erfassung, Ausfälle von Sensoren oder fehlerhafte Datenübertragung. Fehlenden Werte können zu Problemen führen, da sie Metriken und Algorithmen beeinflussen und viele nicht damit umgehen können. 

Es ist wichtig, den Unterschied zwischen `0` und `NULL` zu verstehen, da `NULL` für einen fehlenden Wert steht, während `0` eine tatsächliche Messung von 0 bedeutet.

Die Löschung der Nullwerte ist nur bei sehr geringem Anteil sinnvoll, da es zu einem Informationsverlust führt. Besser ist es, fehlende Werte zu imputieren (d.h. zu schätzen oder zu ergänzen), um die Datenanalyse zu optimieren. Es gibt verschiedene Imputationsmethoden, wie z.B. das Füllen von fehlenden Werten durch den Mittelwert, Median oder Mode der vorhandenen Werte oder die Verwendung von Machine-Learning-Modellen zur Vorhersage der fehlenden Werte auf der Grundlage der verfügbaren Daten. 

*Es ist wichtig, die Imputationsmethoden sorgfältig auszuwählen und zu validieren, um sicherzustellen, dass die Ergebnisse der Datenanalyse nicht verzerrt werden!*

Kontrolle der NULL-Values:

In [14]:
df.isnull().sum()

Messstelle      0
GW-Nummer       0
Datum           0
RASTERVALU      0
HYDROGEOL3     56
O2            475
NO3             0
dtype: int64

### 2.2.1 Nan Werte in numerische Spalten

Für die numerische Spalte 'O2' werden die fehlenden Werte durch den Mittelwert ersetzt.

Dies passiert mit dem <span style="color:blue">**"fillna()"**</span>-Befehl für Pandas-Dataframes.

> Zur Info: Falls fehlende Werte durch beispielsweise "-999" dargestellt sind, können Sie sie mit dem Befehl `replace('alter_Wert', 'neuer_Wert')` ersetzen. Dieser Befehl ersetzt alle Vorkommen des alten Werts durch den neuen Wert im DataFrame.


In [15]:
#  Ersetzt alle NaN-Werte mit dem Mittelwert dieser Spalte
df['O2'] = df['O2'].fillna(df['O2'].mean())

print(df.isnull().sum())

Messstelle     0
GW-Nummer      0
Datum          0
RASTERVALU     0
HYDROGEOL3    56
O2             0
NO3            0
dtype: int64


### 2.2.2 NAN Werte in kategorische Spalten
Einsetzen des meist vorkommenden Wertes, der sog. "Modus"

In [16]:
# Bestimmung des Modus der Spalte "HYDROGEOL3"
hy_mode = df['HYDROGEOL3'].mode()

# Ausgabe der Häufigkeitsverteilung der Werte in der Spalte "HYDROGEOL3" und des Modus
print(df['HYDROGEOL3'].value_counts(), '\n\nModus:', hy_mode)


HYDROGEOL3
Quartäre Kiese und Sande (GWL)         11955
Paläozoikum, Kristallin (GWG)             50
Tertiär im Oberrheingraben (GWG)          37
Oberjura (Raurasische Fazies) (GWL)       35
Oberer Muschelkalk (GWL)                  26
Unterjura und Mitteljura (GWG)            19
Name: count, dtype: int64 

Modus: 0    Quartäre Kiese und Sande (GWL)
Name: HYDROGEOL3, dtype: object


In [17]:
# Ersetze fehlende Werte durch den Modus der Spalte HYDROGEOL3
df['Hydrogeologie'] = df['HYDROGEOL3'].fillna(df['HYDROGEOL3'].mode()[0])
# Entferne die alte Spalte HYDROGEOL3 aus dem Datensatz
df.drop(columns='HYDROGEOL3', inplace=True)


#### Kontrolle

In [18]:
df.isnull().sum()

Messstelle       0
GW-Nummer        0
Datum            0
RASTERVALU       0
O2               0
NO3              0
Hydrogeologie    0
dtype: int64

## 2.3 Feature Aggregation

Wir möchten für jede Messstelle aggregieren:

- Numerische Features (NO3, O2) werden durch den Mittelwert für jede Messstelle aggregiert, während für kategorische Variablen der Modus (häufigst auftretender Wert) verwendet wird.
  
- Dies wird mit der Funktion `groupby()` durchgeführt.

- Die Funktion `agg()` ermöglicht es, verschiedene Aggregationsfunktionen auf unterschiedliche Spalten anzuwenden.
  
- Nach dem Aggregieren wird der Index zurückgesetzt, da andernfalls die Messstellen den Index bilden.
  
- Die beiden aggregierten Datensätze werden dann wieder zusammengeführt.

In [19]:
# Aggregation von numerischen Spalten durch Mittelwert und von kategorischen Spalten durch Modus
GWM = df.groupby('Messstelle').agg({
    'NO3': 'mean',
    'O2': 'mean',
    'Hydrogeologie': pd.Series.mode,
    'RASTERVALU': pd.Series.mode
})

# Zurücksetzen des Index und Umbenennung der Spalte für die Messstelle
GWM.reset_index(inplace=True)

# Umbenennung der Spaltennamen für die kategorischen Variablen
GWM = GWM.rename(columns={'Hydrogeologie': 'Modus_Hydrogeologie', 'RASTERVALU': 'Modus_RASTERVALU'})
GWM

,Messstelle,NO3,O2,Modus_Hydrogeologie,Modus_RASTERVALU
0,"BBR 1 Firma Schultis, Riegel",6.259259,1.315473,Quartäre Kiese und Sande (GWL),121
1,"BBR 2 Firma Thieme, Teningen",1.452941,0.823723,Quartäre Kiese und Sande (GWL),121
2,"BBR 2 Kronenwiese Firma Burda Werk 1, Offenburg",5.600000,1.984615,Quartäre Kiese und Sande (GWL),121
3,"BBR 3186 im Garten der alten Schule, Langhurst",6.968750,0.730346,Quartäre Kiese und Sande (GWL),112
4,"BBR 998 A Kehlerstraße, Neuried-Auenheim",56.164286,1.116071,Quartäre Kiese und Sande (GWL),112
...,...,...,...,...,...
502,"TB Winzergenossenschaft, Sasbach am Kaiserstuhl",23.231034,2.691992,Quartäre Kiese und Sande (GWL),112
503,"TB Wäldeleacker Feldkirch, Hartheim am Rhein",84.274074,7.785307,Quartäre Kiese und Sande (GWL),211
504,"TB1 Glockenackern, Buggingen",59.056667,7.399035,Quartäre Kiese und Sande (GWL),211
505,"TB1 Weingartenhöfe, Forchheim",69.444000,4.908000,Quartäre Kiese und Sande (GWL),211


## 2.4. Train/Test Split

Das Aufteilen des Datensatzes erfolgt mit der Funktion 
<span style="color:blue"></span> ["train_test_split"](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) des Packages sklearn.


Hier kann mit dem Parameter "test_size" das Verhältnis festgelegt werden und mit dem Parameter "random_state" die Reproduzierbarkeit garantiert werden. 

In [20]:
from sklearn.model_selection import train_test_split

# Teilt GWM in Test und Train Datensatz
train, test = train_test_split(GWM, test_size=0.8, random_state=43)

print('Größe Trainingsdaten: ', train.shape)
print('Größe Testdaten: ', test.shape)

Größe Trainingsdaten:  (101, 5)
Größe Testdaten:  (406, 5)


## 2.5. Skalieren
Das Skalieren von Daten vor der Verwendung in einem Machine-Learning-Modell hilft, die Leistung des Modells zu verbessern, die Berechnungskosten zu reduzieren und Verzerrungen zu vermeiden. Es gibt verschiedene Skalierungsmethoden, darunter Min-Max-Skalierung, Standardisierung und logarithmische Skalierung. Die beste Methode hängt von den Daten und dem Modell ab.

### 2.5.1 Normieren
Normieren ist ein Verfahren, bei dem Daten auf eine gemeinsame Einheitsnorm gebracht werden, indem sie durch die Länge des Vektors dividiert werden. Dadurch wird die Richtung der Daten beibehalten, aber die genaue Skala spielt keine Rolle mehr

#### 2.5.1.1 Händisches Skalieren mit Hilfe der Formel:
$X_{norm} = \frac{X - X_{min}}{X_{max} - X_{min}}$

In [21]:
# Minimum und Range der Spalte NO3 im Trainingsdatensatz berechnen
X_min = train.NO3.min()
X_range = train.NO3.max() - train.NO3.min()

# Spalte NO3 im Trainingsdatensatz normalisieren und als neue Spalte hinzufügen
train['NO3_norm'] = (train['NO3'] - X_min) / X_range

# Spalte NO3 im Testdatensatz normalisieren und als neue Spalte hinzufügen, 
# basierend auf den Werten des Trainingsdatensatzes, um Datenleckage zu vermeiden
test['NO3_norm'] = (test['NO3'] - X_min) / X_range

# Ausgabe der statistischen Zusammenfassung der normalisierten Spalte NO3 im Trainingsdatensatz
print(train['NO3_norm'].describe()[['min', 'max', 'mean']])

min     0.000000
max     1.000000
mean    0.204514
Name: NO3_norm, dtype: float64


#### 2.5.1.1 Vorhandene Funktion von sklearn "MinMaxScaler" verwenden

Die Scaler werden zuerst als eingenes Objekt erstellt und müssen dann an Daten gefittet werden.

Anschließend können sie Daten auf Grundlage der Fit-Daten transformieren, hier sklieren.

In [22]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler-Objekt erstellen und auf Trainingsdatensatz anpassen
scaler = MinMaxScaler()
scaler.fit(train[['NO3']])

# Spalte NO3 im Trainings- und Testdatensatz normalisieren und als neue Spalte hinzufügen
train['NO3_norm'] = scaler.transform(train[['NO3']])
test['NO3_norm'] = scaler.transform(test[['NO3']])

print(train['NO3_norm'].describe()[['min', 'max', 'mean']])

min     0.000000
max     1.000000
mean    0.204514
Name: NO3_norm, dtype: float64


### 2.5.2 Standardisieren
Standardisierung ist ein Verfahren, bei dem Daten so transformiert werden, dass sie eine Standardnormalverteilung mit einem Mittelwert von 0 und einer Standardabweichung von 1 haben. Die Standardisierung erfolgt durch Subtraktion des Mittelwerts der Daten und Division durch die Standardabweichung der Daten. Dadurch werden die Daten zentriert und die Skala der Daten wird angepasst. Die Standardisierung kann helfen, Ausreißer zu reduzieren und die Modellleistung zu verbessern, insbesondere bei linearen Modellen wie der linearen Regression.

#### 2.5.2.1 Händisches Skalieren mit Hilfe der Formel:
$X_{std} = \frac{X - mean(X)}{stdw(X)}$

In [23]:
# Berechnen von Mittelwert und Standardabweichung der NO3-Spalte im Trainingsdatensatz
X_mean = train['NO3'].mean()
X_std = train['NO3'].std()

# Erstellen einer neuen Spalte im Trainings- und Testdatensatz durch Standardisierung der NO3-Spalte,
# basierend auf dem Mittelwert und der Standardabweichung des Trainingsdatensatzes, 
# um Datenleckage zu vermeiden
train['NO3_std'] = (train['NO3'] - X_mean) / X_std
test['NO3_std'] = (test['NO3'] - X_mean) / X_std

# Ausgabe des Mittelwerts und der Standardabweichung der standardisierten Spalte NO3_std im Trainingsdatensatz
print('mean(X): ', train['NO3_std'].mean())
print('std(X): ', train['NO3_std'].std())


mean(X):  5.27630744376312e-17
std(X):  0.9999999999999999


#### 2.5.2.2 Vorhandene Funktion von sklearn "StandardScaler"

In [24]:
from sklearn.preprocessing import StandardScaler

# Erstellen eines StandardScaler-Objekts
scaler = StandardScaler()

# Anpassen des StandardScaler-Objekts an den Trainingsdatensatz
scaler.fit(train[['NO3']])

# Transformieren der NO3-Spalte im Trainings- und Testdatensatz mithilfe des StandardScaler-Objekts
train['NO3_std'] = scaler.transform(train[['NO3']])
test['NO3_std'] = scaler.transform(test[['NO3']])

# Ausgabe des Mittelwerts und der Standardabweichung der standardisierten Spalte NO3_std im Trainingsdatensatz
print('mean(X): ', train['NO3_std'].mean())
print('std(X): ', train['NO3_std'].std())


mean(X):  5.386230515508185e-17
std(X):  1.004987562112089


In [25]:
train.head()

,Messstelle,NO3,O2,Modus_Hydrogeologie,Modus_RASTERVALU,NO3_norm,NO3_std
49,"BR 7 Hofgut Freudenberg, Heddesheim",0.366000,0.394000,Quartäre Kiese und Sande (GWL),211,0.001255,-0.847277
298,"GWM E40 ZWK Kurpfalz, Schwetzingen",15.027586,5.883984,Quartäre Kiese und Sande (GWL),112,0.140704,-0.265988
437,TB 2 Transportbeton Freiburg,16.500000,6.939286,Quartäre Kiese und Sande (GWL),121,0.154709,-0.207611
441,"TB 3 Firma Hansa Heemann AG, Bruchsal",0.287037,0.546296,Quartäre Kiese und Sande (GWL),121,0.000504,-0.850407
382,"GWM Tief, Heddesheim",0.250000,0.347619,Quartäre Kiese und Sande (GWL),211,0.000151,-0.851876


In [26]:
test.head()

,Messstelle,NO3,O2,Modus_Hydrogeologie,Modus_RASTERVALU,NO3_norm,NO3_std
498,"TB WW Neuburgweier, Rheinstetten",0.664062,1.625000,Quartäre Kiese und Sande (GWL),231,0.004090,-0.835459
328,"GWM K 20 Kehl, Goldscheuer",0.227778,0.255556,Quartäre Kiese und Sande (GWL),231,-0.000060,-0.852757
321,"GWM G0/1 KIT, Leopoldshafen",6.945455,3.259091,Quartäre Kiese und Sande (GWL),313,0.063833,-0.586421
170,"GWM 1279 A, Oberhausen",14.138462,7.215385,Quartäre Kiese und Sande (GWL),231,0.132248,-0.301239
432,"TB 116 Universität Albertstraße, Freiburg",14.133333,7.412369,Quartäre Kiese und Sande (GWL),121,0.132199,-0.301442


## 2.6 Encodieren
Encodieren ist die Konvertierung von Daten in eine geeignete Darstellung, die von einer Maschine/Computer verarbeitet werden kann. Dies kann die Umwandlung von Daten in numerische, binäre oder kategoriale Darstellungen umfassen, je nachdem, welche Art von Daten verarbeitet werden sollen und welche Anforderungen das Modell hat, das mit den Daten trainiert oder verwendet wird. 

*Die Wahl der richtigen Encoding-Methode kann einen erheblichen Einfluss auf die Leistung eines Machine-Learning-Modells haben*

### 2.6.1 Binäres Encodieren
Jede Kategorie wird umgewandelt in Zahlenwert

Der 3-stellige Corine Code ist eine Stufenweise Beschreibung der Landnutzung, wobei die erste Ziffer die gröbste Einordnung darstellt.
- 1XX = Artificial Surfaces 
- 2XX = Agricultural areas 
- 3XX = Forest and seminatural areas
- 4XX = Wetlands 
- 5XX = Water bodies 

Im Folgenden erfolgt ein binäres Encoding nach der Klasse 1.
Dafür wird zuerst eine neue Spalte erstellt, die nur die erste Ziffer enthät und anschließend in Abhängigkeit davon ein binäres Encoding.


In [27]:
datasets = [train, test]

for dataset in datasets:
    # Extrahiert erste Ziffer in "Modus_RASTERVALU"
    dataset['Corine'] = dataset['Modus_RASTERVALU'].apply(lambda x: int(str(x)[0]))
    
    # Falls erste Ziffer ungleich 1, dann 0
    dataset['Artificial_Surface'] = dataset['Corine'].apply(lambda x: 1 if x == 1  else 0)
    
    # Entfernt "Modus_RASTERVALU"
    dataset.drop(columns='Modus_RASTERVALU', inplace=True)


### 2.6.2 One-Hot
Encoding für Nominal-Daten (ohne Rangfolge)  
One-Hot-Encoding ist ein Verfahren zur Darstellung von Kategorien oder Nominaldaten als numerische Vektoren. Dabei wird jeder Kategorie ein eigener Binärwert zugeordnet.

In [28]:
from sklearn.preprocessing import OneHotEncoder

Beispiel für One-Hot Encoding:

In [29]:
from sklearn.preprocessing import OneHotEncoder

# Erstelle einen OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Passe den Encoder auf die "Corine"-Spalte des Trainingsdatensatzes an
encoder.fit(train[['Corine']])

# Wende den Encoder auf die "Corine"-Spalte des Testdatensatzes an und erhalte die One-Hot-kodierten Features
OHE_test = encoder.transform(test[['Corine']])

# Konvertiere die One-Hot-kodierten Features in ein DataFrame und benenne die Spalten entsprechend
OHE_test = pd.DataFrame(OHE_test, columns=encoder.get_feature_names_out(['Corine']))
OHE_test

,Corine_1,Corine_2,Corine_3,Corine_5
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
401,1.0,0.0,0.0,0.0
402,0.0,0.0,1.0,0.0
403,0.0,1.0,0.0,0.0
404,1.0,0.0,0.0,0.0


### 2.6.3 Target Encoding
Der Mittelwert der Ziel Variable wird für jede Kategorie berechnet und als Wert für die entsprechende Kategorie verwendet  
Das Encodieren muss nach dem Data-Split erfolgen um ein Data Leakage zu vermeiden

In [30]:
from category_encoders.target_encoder import TargetEncoder

# Target-Encoder-Objekt erstellen
encoder = TargetEncoder()
# Trainingsdaten kodieren
train['Target_Hy'] = encoder.fit_transform(train['Modus_Hydrogeologie'], train['NO3'])
# Testdaten kodieren
test['Target_Hy'] = encoder.transform(test['Modus_Hydrogeologie'])

# Ausgabe der ersten Zeilen der Trainingsdaten mit der neuen Spalte 'Target_Hy'
train.head()



,Messstelle,NO3,O2,Modus_Hydrogeologie,NO3_norm,NO3_std,Corine,Artificial_Surface,Target_Hy
49,"BR 7 Hofgut Freudenberg, Heddesheim",0.366000,0.394000,Quartäre Kiese und Sande (GWL),0.001255,-0.847277,2,0,21.923784
298,"GWM E40 ZWK Kurpfalz, Schwetzingen",15.027586,5.883984,Quartäre Kiese und Sande (GWL),0.140704,-0.265988,1,1,21.923784
437,TB 2 Transportbeton Freiburg,16.500000,6.939286,Quartäre Kiese und Sande (GWL),0.154709,-0.207611,1,1,21.923784
441,"TB 3 Firma Hansa Heemann AG, Bruchsal",0.287037,0.546296,Quartäre Kiese und Sande (GWL),0.000504,-0.850407,1,1,21.923784
382,"GWM Tief, Heddesheim",0.250000,0.347619,Quartäre Kiese und Sande (GWL),0.000151,-0.851876,2,0,21.923784


## 2.7. Speichern der Ergebnisse

Um einen Datensatz zu speichern,ist es sinnvoll diesen als csv.Datei zu exportieren.

Das Exportieren erfolgt über den Befehl ".to_csv" für Dataframes.

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [31]:
train.to_csv('res.csv')

# 3. Übung

Bereite jetzt selbst den Datensatz "Nitratmessungen_aufgabe.csv" auf:
1. Ersetze die Werte unter der NWG.
2. Ersetze alle NULL-Values.
3. Aggregiere die Daten für jede .
4. Splitte die Daten (85:15).
5. Standardisiere die numerischen Variablen.
6. Encode die Landnutzung binär nach der Landwirtschaft (Corine = 2XX) --> vorhanden = 1, nicht vorhanden = 0.
7. Target-Encode die Hyrogeologie auf Basis der Nitratkonzentration.
8. Speichere deine Ergebnisse.
9. Fertig! :).

Tipps:
- für die Features der Sauerstoff-Konzentration und der Hydrogeologie liegen die NULL-Values nicht als diese direkt vor, hier ist es hilfreich mal die Features mit ".describe()" bzw. ".unique()" zu betrachten.